<img src="https://deecamp.chuangxin.com/assets/image/logo_nav_zh.jpg" width="40%">

In [2]:
import pandas as pd
import csv

In [16]:
! ls /data/jupyter_root/ZJ/test1.0 

block_1.attributes  block_2.attributes	block_3.attributes
block_1.tuples	    block_2.tuples	block_3.tuples


## 定义一个函数，先输出原始feature文件的P & R，再输出通过D-Cube分块的P & R

名称 | 定义
 --- | ---
TP	| 真实类别为positive，模型预测的类别也为positive
FP	| 预测为positive，但真实类别为negative，真实类别和预测类别不一致
FN	| 预测为negative，但真实类别为positive，真实类别和预测类别不一致
TN	| 真实类别为negative，模型预测的类别也为negative

准确率（accuracy）计算公式如下所示 不计算这个

\begin{equation}\label{equ:accuracy} \mbox{accuracy} = \frac{TP+TN}{TP+TN+FP+FN} = \frac{TP+TN }{\mbox{all data}} \end{equation}

positive class的精确率（precision）计算公式如下

\begin{equation}\label{equ:precision} \mbox{precision} = \frac{TP}{TP+FP} = \frac{TP}{\mbox{预测为positive的样本}} \end{equation}

positive class的召回率（recall）计算公式如下

\begin{equation}\label{equ:recall} \mbox{recall} = \frac{TP}{TP+FN} = \frac{TP}{\mbox{真实为positive的样本}} \end{equation}


# 得到feature have name的PR（输入：f_count：第几个特征，j：无用，choice：第几个block）

In [4]:
def get_PR(f_count,j,choice):#f_count：the index of feature;j: index of test; choice: index of block
    
    #准备用于读取数据的 pandas头（防止第一行数据成为标签）
    data_path="/data/jupyter_root/dcube_data/have_name/feature"#feature文件位置
    #try:
    cols = pd.read_csv(data_path+str(f_count)+"/block_1.tuples",nrows=1).columns#先读第一行，用于提取数据维度
    #except e:
    #    print(e)
    #    print("false1")
    #    return 0        

    user_index=['user_name']#初始化一个等待补全的names
    #print(cols.shape[0]-1)
    for i in range(1,cols.shape[0]-1):
            user_index.append(str(i))
    user_index.append('count')#补上最后的count
    
    #开始读取数据
    #result=pd.read_csv(data_path+str(f_count)+"/test"+str(j)+"/block_1.tuples",sep=',',names=user_index)#读取全部的数据，并给上names
    #try:
    if(choice=='all'):#读取blocks.txt
        result=pd.read_csv(data_path+str(f_count)+"/blocks.txt",sep=',',names=user_index)
    elif(choice==1 or choice==2 or choice==3 ):#读取block_choice.tuples
        result=pd.read_csv(data_path+str(f_count)+"/block_"+str(choice)+".tuples",sep=',',names=user_index)
    elif(choice==12 or choice==13 or choice==23 or choice ==123 ):#读取blockchoice.txt
        result=pd.read_csv(data_path+str(f_count)+"/block"+str(choice)+".txt",sep=',',names=user_index)
    #except:
    #    print("false2")
    #    return 0
    names = result['user_name'].drop_duplicates()#预测出坏用户的用户列表
    #print(names)
    dataset = pd.read_csv(open('/data/csv/label_eventsV1.csv','r',encoding = 'gb18030'))#读取总标签数据
    data = dataset[dataset['user_name'].isin(names)][['user_name','label']]#预测出坏用户的真实标签分布
    data = data.drop_duplicates()#去重
    #print(data)

    
    raw_result=pd.read_csv(data_path+str(f_count)+".txt",sep=',',names=user_index)#读取全部的原始数据，并给上names
    raw_names = raw_result['user_name'].drop_duplicates()#原始数据中用户的用户列表
    #print(raw_names)
    raw_data = dataset[dataset['user_name'].isin(raw_names)][['user_name','label']]#原始数据中坏用户的真实标签分布
    raw_data = raw_data.drop_duplicates()#去重
    #print(raw_data)
    
    
    TP=data.label.sum()
    if(TP==0):
        return 0
    P=raw_data.label.sum()
    FP=data.label.count()-data.label.sum()
    N=raw_data.label.count()-raw_data.label.sum()
    Precision = (TP/P)/(TP/P+FP/N) #精确率计算

    #data_label_1 = dataset[dataset['label']==1][['user_name','label']] #原数据中label为1的坏用户
    #data_label_1 = data_label_1.drop_duplicates()#去重
    Recall = data.label.sum()/raw_data.label.sum() # 召回率计算
    print("TP",TP,"P",P,"FP",FP,"N",N)
    print('测试特征'+str(f_count)+"_"+str(j)+str(choice))
    print('\t\t精确率 Precision\t= '+str(Precision))
    print('\t\t召回率 Recall   \t= '+str(Recall))
#get_PR(1,1)

In [9]:
def get_no_name_PR(f_count,feature_user,find_user):
    
    dataset = pd.read_csv(open('/data/csv/label_eventsV1.csv','r',encoding = 'gb18030'))#读取总标签数据
    data = dataset[dataset['user_name'].isin(find_user)][['user_name','label']]#预测出坏用户的真实标签分布
    data = data.drop_duplicates()#去重
    #print(data)

    
    #raw_result=pd.read_csv(data_path+str(f_count)+".txt",sep=',',names=user_index)#读取全部的原始数据，并给上names
    #raw_names = raw_result['user_name'].drop_duplicates()#原始数据中用户的用户列表
    #print(raw_names)
    raw_data = dataset[dataset['user_name'].isin(feature_user)][['user_name','label']]#原始数据中坏用户的真实标签分布
    raw_data = raw_data.drop_duplicates()#去重
    #print(raw_data)
    
    
    TP=data.label.sum()
    #if(TP==0):
        #return 0
    P=raw_data.label.sum()
    FP=data.label.count()-data.label.sum()
    N=raw_data.label.count()-raw_data.label.sum()
    Precision = (TP/P)/(TP/P+FP/N) #精确率计算

    #data_label_1 = dataset[dataset['label']==1][['user_name','label']] #原数据中label为1的坏用户
    #data_label_1 = data_label_1.drop_duplicates()#去重
    Recall = data.label.sum()/raw_data.label.sum() # 召回率计算
    print("TP",TP,"P",P,"FP",FP,"N",N)
    print('测试特征'+f_count)
    print('\t\t精确率 Precision\t= '+str(Precision))
    print('\t\t召回率 Recall   \t= '+str(Recall))

In [11]:
import re
result_list=pd.read_csv("/data/jupyter_root/dcube_data/no_name_add/result_list2_811.csv")
#print(result_list)
def get_set(s):
    s=re.sub('[ {}]','',s).split(',')
    return set(map(int,s))
for i in range(45):
    try:
        al=get_set(result_list.iloc[i,0])
        get_no_name_PR("feature"+str(i)+'block_1',al,get_set(result_list.iloc[i,1]))
        get_no_name_PR("feature"+str(i)+'block_2',al,get_set(result_list.iloc[i,2]))
        get_no_name_PR("feature"+str(i)+'block_3',al,get_set(result_list.iloc[i,3]))
        #get_no_name_PR("feature"+str(i)+'block_12',al,get_set(result_list.iloc[i,4]))
        #get_no_name_PR("feature"+str(i)+'block_123',al,get_set(result_list.iloc[i,5]))
    except:
        continue
        print('false')
#for i in range

TP 5498 P 5538 FP 3855 N 3871
测试特征feature0block_1
		精确率 Precision	= 0.49922320801858744
		召回率 Recall   	= 0.9927771758757674
TP 4800 P 5538 FP 2608 N 3871
测试特征feature0block_2
		精确率 Precision	= 0.5626469844389083
		召回率 Recall   	= 0.866738894907909
TP 5538 P 5538 FP 3871 N 3871
测试特征feature0block_3
		精确率 Precision	= 0.5
		召回率 Recall   	= 1.0
TP 803 P 5538 FP 478 N 3871
测试特征feature1block_1
		精确率 Precision	= 0.5400697445277858
		召回率 Recall   	= 0.14499819429396893
TP 780 P 5538 FP 480 N 3871
测试特征feature1block_2
		精确率 Precision	= 0.5318038192059349
		召回率 Recall   	= 0.14084507042253522
TP 734 P 5538 FP 501 N 3871
测试特征feature1block_3
		精确率 Precision	= 0.505945313373643
		召回率 Recall   	= 0.13253882267966774
TP 115 P 5538 FP 67 N 3871
测试特征feature2block_1
		精确率 Precision	= 0.5454043133454463
		召回率 Recall   	= 0.02076561935716865
TP 107 P 5538 FP 53 N 3871
测试特征feature2block_2
		精确率 Precision	= 0.5852629109905032
		召回率 Recall   	= 0.01932105453232214
TP 93 P 5538 FP 66 N 3871
测试特征feature2block_3


TP 3855 P 4800 FP 2034 N 2608
测试特征feature22block_3
		精确率 Precision	= 0.5073330830436836
		召回率 Recall   	= 0.803125
TP 3690 P 4800 FP 1900 N 2608
测试特征feature23block_1
		精确率 Precision	= 0.5134318420446106
		召回率 Recall   	= 0.76875
TP 4496 P 4800 FP 2369 N 2608
测试特征feature23block_2
		精确率 Precision	= 0.5076713763588049
		召回率 Recall   	= 0.9366666666666666
TP 4068 P 4800 FP 2137 N 2608
测试特征feature23block_3
		精确率 Precision	= 0.5084282585892789
		召回率 Recall   	= 0.8475
TP 4404 P 4800 FP 2283 N 2608
测试特征feature24block_1
		精确率 Precision	= 0.5117454831645223
		召回率 Recall   	= 0.9175
TP 4560 P 4800 FP 2426 N 2608
测试特征feature24block_2
		精确率 Precision	= 0.505261440574272
		召回率 Recall   	= 0.95
TP 1388 P 4800 FP 819 N 2608
测试特征feature24block_3
		精确率 Precision	= 0.4793873849439764
		召回率 Recall   	= 0.2891666666666667
TP 5334 P 5484 FP 3734 N 3828
测试特征feature25block_1
		精确率 Precision	= 0.4992822736256978
		召回率 Recall   	= 0.9726477024070022
TP 192 P 5484 FP 131 N 3828
测试特征feature25block_2
		精确率 Precis

In [6]:
for i in range(51,136):
    #for j in range(51,136):
        j=3
        try:
            #if(j==3):
                get_PR(i,j,1)
                get_PR(i,j,2)
                get_PR(i,j,3)
                #get_PR(i,j,'all')
                #get_PR(i,j,12)
                #get_PR(i,j,13)
                #get_PR(i,j,23)
                #get_PR(i,j,123)
        except:
            print("false")
            continue

TP 5538 P 5538 FP 3871 N 3871
测试特征51_31
		精确率 Precision	= 0.5
		召回率 Recall   	= 1.0
TP 5538 P 5538 FP 3871 N 3871
测试特征51_32
		精确率 Precision	= 0.5
		召回率 Recall   	= 1.0
TP 5538 P 5538 FP 3871 N 3871
测试特征51_33
		精确率 Precision	= 0.5
		召回率 Recall   	= 1.0
TP 2032 P 5538 FP 1352 N 3871
测试特征52_31
		精确率 Precision	= 0.5123262517481643
		召回率 Recall   	= 0.3669194655110148
TP 3119 P 5538 FP 2238 N 3871
测试特征52_32
		精确率 Precision	= 0.49345269290406746
		召回率 Recall   	= 0.563199711087035
TP 318 P 5538 FP 274 N 3871
测试特征52_33
		精确率 Precision	= 0.4478905832141727
		召回率 Recall   	= 0.05742145178764897
TP 1038 P 5538 FP 720 N 3871
测试特征53_31
		精确率 Precision	= 0.5019198152060756
		召回率 Recall   	= 0.18743228602383533
TP 1748 P 5538 FP 1151 N 3871
测试特征53_32
		精确率 Precision	= 0.5149257127411184
		召回率 Recall   	= 0.3156374142289635
TP 44 P 5538 FP 18 N 3871
测试特征53_33
		精确率 Precision	= 0.6308109389351426
		召回率 Recall   	= 0.007945106536655833
TP 1153 P 5538 FP 821 N 3871
测试特征54_31
		精确率 Precision	= 0.49536984

TP 2066 P 5484 FP 1535 N 3828
测试特征75_33
		精确率 Precision	= 0.48440298732349957
		召回率 Recall   	= 0.3767323121808899
TP 3 P 5484 FP 36 N 3828
测试特征76_31
		精确率 Precision	= 0.05497156643115629
		召回率 Recall   	= 0.0005470459518599562
TP 1797 P 5484 FP 1273 N 3828
测试特征76_32
		精确率 Precision	= 0.4963125668828852
		召回率 Recall   	= 0.3276805251641138
TP 3043 P 5484 FP 2077 N 3828
测试特征76_33
		精确率 Precision	= 0.5056065244860946
		召回率 Recall   	= 0.5548869438366156
TP 1 P 5484 FP 38 N 3828
测试特征77_31
		精确率 Precision	= 0.01803788521345773
		召回率 Recall   	= 0.00018234865061998541
TP 236 P 5484 FP 225 N 3828
测试特征77_32
		精确率 Precision	= 0.42268498503725244
		召回率 Recall   	= 0.043034281546316555
TP 1757 P 5484 FP 1203 N 3828
测试特征77_33
		精确率 Precision	= 0.5048241213271917
		召回率 Recall   	= 0.32038657913931434
TP 57 P 5484 FP 91 N 3828
测试特征78_31
		精确率 Precision	= 0.30421616195415757
		召回率 Recall   	= 0.010393873085339168
TP 1398 P 5484 FP 995 N 3828
测试特征78_32
		精确率 Precision	= 0.4951408773622508
		召回率 Recal

TP 257 P 5538 FP 307 N 3871
测试特征101_33
		精确率 Precision	= 0.36914367388951375
		召回率 Recall   	= 0.046406644998194295
TP 5447 P 5538 FP 3643 N 3871
测试特征102_31
		精确率 Precision	= 0.511032440867875
		召回率 Recall   	= 0.9835680751173709
TP 719 P 5538 FP 1282 N 3871
测试特征102_32
		精确率 Precision	= 0.281620849613451
		召回率 Recall   	= 0.12983026363308053
TP 5 P 5538 FP 26 N 3871
测试特征102_33
		精确率 Precision	= 0.1184929871497401
		召回率 Recall   	= 0.0009028530155290719
TP 5466 P 5538 FP 3660 N 3871
测试特征103_31
		精确率 Precision	= 0.5107391948714064
		召回率 Recall   	= 0.9869989165763814
TP 103 P 5538 FP 490 N 3871
测试特征103_32
		精确率 Precision	= 0.12810743580458775
		召回率 Recall   	= 0.018598772119898882
TP 412 P 5538 FP 314 N 3871
测试特征103_33
		精确率 Precision	= 0.4783909215474068
		召回率 Recall   	= 0.07439508847959553
TP 5334 P 5484 FP 3734 N 3828
测试特征104_31
		精确率 Precision	= 0.4992822736256978
		召回率 Recall   	= 0.9726477024070022
TP 195 P 5484 FP 133 N 3828
测试特征104_32
		精确率 Precision	= 0.5057892768282569
		召回率 R

TP 1200 P 4800 FP 776 N 2608
测试特征126_33
		精确率 Precision	= 0.4565826330532212
		召回率 Recall   	= 0.25
TP 1410 P 4800 FP 774 N 2608
测试特征127_31
		精确率 Precision	= 0.4974352314784754
		召回率 Recall   	= 0.29375
TP 3555 P 4800 FP 1951 N 2608
测试特征127_32
		精确率 Precision	= 0.49749520289500454
		召回率 Recall   	= 0.740625
TP 16 P 4800 FP 10 N 2608
测试特征127_33
		精确率 Precision	= 0.46504992867332384
		召回率 Recall   	= 0.0033333333333333335
TP 4285 P 4800 FP 2294 N 2608
测试特征128_31
		精确率 Precision	= 0.5036977474570099
		召回率 Recall   	= 0.8927083333333333
TP 2141 P 4800 FP 1190 N 2608
测试特征128_32
		精确率 Precision	= 0.49432210124039816
		召回率 Recall   	= 0.44604166666666667
TP 671 P 4800 FP 420 N 2608
测试特征128_33
		精确率 Precision	= 0.4646794662089535
		召回率 Recall   	= 0.13979166666666668
TP 4245 P 4800 FP 2264 N 2608
测试特征129_31
		精确率 Precision	= 0.5046439628482973
		召回率 Recall   	= 0.884375
TP 2196 P 4800 FP 1227 N 2608
测试特征129_32
		精确率 Precision	= 0.4930087266957557
		召回率 Recall   	= 0.4575
TP 524 P 4800 FP 312 N

In [51]:
#特征取交集测试
get_PR(57,3,1)
get_PR(57,3,2)
import pandas as pd
dataset = pd.read_csv(open('/data/csv/label_eventsV1.csv','r',encoding = 'gb18030'))#读取总标签数据
user_index=['user_name']#初始化一个等待补全的names
#print(cols.shape[0]-1)
for i in range(1,3):
        user_index.append(str(i))
user_index.append('count')
user_index.append('now')
result_1=pd.read_csv("/data/jupyter_root/dcube_data/feature57/block_1.tuples",sep=',',names=user_index)

#user_index.pop()
result_2=pd.read_csv("/data/jupyter_root/dcube_data/feature57/block_2.tuples",sep=',',names=user_index)
names=set(result_1['user_name'].drop_duplicates().values)&set(result_2['user_name'].drop_duplicates().values)
data = dataset[dataset['user_name'].isin(names)][['user_name','label']]#预测出坏用户的真实标签分布
data = data.drop_duplicates()

#user_index.append('now')
raw_result=pd.read_csv("/data/jupyter_root/dcube_data/feature57.txt",sep=',',names=user_index)#读取全部的原始数据，并给上names
raw_names = raw_result['user_name'].drop_duplicates()#原始数据中用户的用户列表
#print(raw_names)
raw_data = dataset[dataset['user_name'].isin(raw_names)][['user_name','label']]#原始数据中用户的真实标签分布
raw_data = raw_data.drop_duplicates()#去重

TP=data.label.sum()
P=raw_data.label.sum()
FP=data.label.count()-data.label.sum()
N=raw_data.label.count()-raw_data.label.sum()
Precision = (TP/P)/(TP/P+FP/N) #精确率计算


    #data_label_1 = dataset[dataset['label']==1][['user_name','label']] #原数据中label为1的坏用户
    #data_label_1 = data_label_1.drop_duplicates()#去重
Recall = data.label.sum()/raw_data.label.sum() # 召回率计算
print("TP",TP,"P",P,"FP",FP,"N",N)
print('\t\t精确率 Precision\t= '+str(Precision))
print('\t\t召回率 Recall   \t= '+str(Recall))
#user_index.pop()

raw_result=pd.read_csv("/data/jupyter_root/dcube_data/feature57.txt",sep=',',names=user_index)#读取全部的原始数据，并给上names
raw_names = raw_result['user_name'].drop_duplicates()#原始数据中用户的用户列表
#print(raw_names)
raw_data = dataset[dataset['user_name'].isin(raw_names)][['user_name','label']]#原始数据中坏用户的真实标签分布
raw_data = raw_data.drop_duplicates()#去重

TP=data.label.sum()
P=raw_data.label.sum()
FP=data.label.count()-data.label.sum()
N=raw_data.label.count()-raw_data.label.sum()
Precision = (TP/P)/(TP/P+FP/N) #精确率计算

    #data_label_1 = dataset[dataset['label']==1][['user_name','label']] #原数据中label为1的坏用户
    #data_label_1 = data_label_1.drop_duplicates()#去重
Recall = data.label.sum()/raw_data.label.sum() # 召回率计算
print("TP",TP,"P",P,"FP",FP,"N",N)
print('\t\t精确率 Precision\t= '+str(Precision))
print('\t\t召回率 Recall   \t= '+str(Recall))

TP 3244 P 5538 FP 1666 N 3871
测试特征57_31
		精确率 Precision	= 0.5764607439131921
		召回率 Recall   	= 0.5857710364752619
TP 2431 P 5538 FP 2186 N 3871
测试特征57_32
		精确率 Precision	= 0.437358053498462
		召回率 Recall   	= 0.43896713615023475
TP 172 P 5538 FP 160 N 3871
		精确率 Precision	= 0.4290324326692837
		召回率 Recall   	= 0.031058143734200072
TP 172 P 5538 FP 160 N 3871
		精确率 Precision	= 0.4290324326692837
		召回率 Recall   	= 0.031058143734200072


# 精确率Precision的计算

# 召回率Recall的计算

In [5]:
pd.set_option('max_row',200) 
pd.set_option('display.max_colwidth', 100)
pd.read_csv("/data/jupyter_root/dcube_data/have_name/feature_list.csv",sep=',')

,Unnamed: 0,Feature_combination,Standard,Poly,user_name_dic,user_name_num
0,feature1,['user_name'],user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
1,feature2,"['user_name', 'time_stamp_day']",user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
2,feature3,"['user_name', 'time_stamp_hour']",user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
3,feature4,"['user_name', 'time_stamp_3hour']",user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
4,feature5,"['user_name', 'time_stamp_6hour']",user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
5,feature6,['user_name'],os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
6,feature7,"['user_name', 'time_stamp_day']",os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
7,feature8,"['user_name', 'time_stamp_hour']",os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
8,feature9,"['user_name', 'time_stamp_3hour']",os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
9,feature10,"['user_name', 'time_stamp_6hour']",os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409


In [61]:
pwd

'/data/jupyter_root/LSK'

In [21]:
import pandas as pd
user_index=['user_name']#初始化一个等待补全的names
#print(cols.shape[0]-1)
for i in range(1,3):
        user_index.append(str(i))
user_index.append('count')
result_1=pd.read_csv("./block12.txt",sep=',',names=user_index)
result_2=pd.read_csv("./block13.txt",sep=',',names=user_index)
set(result_1['user_name'].drop_duplicates().values)&set(result_2['user_name'].drop_duplicates().values)
fet

{1343494,
 458762,
 1310734,
 1343510,
 1540121,
 1671197,
 65569,
 32805,
 65598,
 524351,
 393281,
 524359,
 1540168,
 1638472,
 524364,
 65614,
 1540176,
 393300,
 65629,
 1146980,
 262247,
 196712,
 65645,
 360561,
 262258,
 1540214,
 1605754,
 229504,
 1409157,
 393356,
 1540237,
 1310879,
 1376418,
 1376419,
 1540260,
 262312,
 1671346,
 1179832,
 65722,
 1179837,
 65728,
 393420,
 491735,
 1147095,
 1540313,
 1147101,
 295139,
 1343718,
 1179879,
 327915,
 65779,
 327928,
 262394,
 524541,
 295167,
 1540356,
 1343750,
 1540359,
 1409286,
 1179919,
 1540369,
 65821,
 1507622,
 1540391,
 1311025,
 1179954,
 1147187,
 459060,
 1212727,
 1179962,
 393538,
 1507655,
 1147208,
 1179977,
 1507657,
 1507668,
 1507673,
 1540452,
 1507685,
 1540453,
 1671524,
 1507689,
 1507693,
 98670,
 1507698,
 1343865,
 196993,
 1507716,
 1507719,
 98696,
 524681,
 1507724,
 1573265,
 1507730,
 655763,
 197018,
 131491,
 1540516,
 393640,
 524723,
 1507763,
 1507769,
 131514,
 491981,
 1507792,
 15077

In [27]:
pd.set_option('max_row',200) 
pd.set_option('display.max_colwidth', 100)
pd.read_csv("/data/jupyter_root/dcube_data/no_name/feature_list.csv",sep=',')

,Unnamed: 0,Feature_combination,Standard,Poly
0,feature1,['time_stamp_day'],user_agent,nunique
1,feature2,['time_stamp_hour'],user_agent,nunique
2,feature3,['time_stamp_3hour'],user_agent,nunique
3,feature4,['time_stamp_6hour'],user_agent,nunique
4,feature5,['time_stamp_day'],os_version,nunique
5,feature6,['time_stamp_hour'],os_version,nunique
6,feature7,['time_stamp_3hour'],os_version,nunique
7,feature8,['time_stamp_6hour'],os_version,nunique
8,feature9,['time_stamp_day'],ip_1,nunique
9,feature10,['time_stamp_hour'],ip_1,nunique


In [12]:
pd.set_option('max_row',200) 
pd.set_option('display.max_colwidth', 100)
pd.read_csv("/data/jupyter_root/dcube_data/no_name_add/feature_list.csv",sep=',')

,Unnamed: 0,Feature_combination,Standard,Poly
0,feature126,['user_agent'],user_name,nunique
1,feature127,"['user_agent', 'time_stamp_day']",user_name,nunique
2,feature128,"['user_agent', 'time_stamp_hour']",user_name,nunique
3,feature129,"['user_agent', 'time_stamp_3hour']",user_name,nunique
4,feature130,"['user_agent', 'time_stamp_6hour']",user_name,nunique
5,feature131,['os_version'],user_name,nunique
6,feature132,"['os_version', 'time_stamp_day']",user_name,nunique
7,feature133,"['os_version', 'time_stamp_hour']",user_name,nunique
8,feature134,"['os_version', 'time_stamp_3hour']",user_name,nunique
9,feature135,"['os_version', 'time_stamp_6hour']",user_name,nunique


In [7]:
pd.set_option('max_row',200) 
pd.set_option('display.max_colwidth', 100)
pd.read_csv("/data/jupyter_root/dcube_data/no_name_add/result_list2_811.csv",sep=',')

,feature_user,block_1.tuples,block_2.tuples,block_3.tuples,block_4.tuples,block_5.tuples
0,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1343494, 458762, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 1671224, 655...","{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167..."
1,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1343494, 245766, 1597450, 319501, 1310734, 221217, 65569, 548900, 1130535, 41012, 1589314, 2212...","{1130498, 466948, 286725, 245766, 319501, 81934, 1310734, 1327123, 516120, 221211, 65569, 548900...","{1130498, 1089538, 466949, 499719, 1310734, 1646615, 65569, 1130535, 1089592, 114747, 1638472, 1...","{466948, 1658884, 546826, 1601547, 1310734, 1548306, 1529874, 118805, 1343510, 544789, 221211, 1...","{1595392, 1089538, 1658884, 245766, 499719, 221192, 348169, 546826, 141325, 1433614, 1556496, 12..."
2,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{20487, 221192, 1514503, 1182730, 76304, 1165329, 1548306, 282649, 521755, 1590812, 160289, 2207...","{280576, 373249, 389632, 102914, 395264, 160268, 319501, 1652756, 520214, 73243, 65569, 1583652,...","{198656, 73218, 117763, 114182, 1604102, 92681, 1573897, 1464845, 1290254, 453138, 1653782, 5207...","{1508352, 461314, 1343494, 245766, 118796, 1464845, 1313814, 471065, 73243, 250396, 118817, 6556...","{1576450, 1090563, 419333, 116742, 519687, 1524743, 118805, 116761, 542748, 475676, 366110, 2150..."
3,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{280576, 395264, 317450, 319501, 81934, 1308689, 1327123, 1652756, 118805, 520214, 1656859, 6556...","{1343494, 1536007, 245766, 1597450, 118796, 319501, 118805, 471065, 1607712, 118817, 65569, 5489...","{245766, 20487, 221192, 1165329, 1548306, 282649, 116788, 1071162, 520253, 1656894, 1052743, 577...","{116742, 1323024, 118805, 1646615, 116761, 542748, 1587229, 215072, 1634346, 479275, 550958, 107...","{198656, 245766, 1290254, 1298455, 215072, 460840, 1177644, 116788, 471108, 71753, 397389, 13067..."
4,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{280576, 1650688, 395264, 286725, 317450, 319501, 81934, 1308689, 1327123, 1652756, 118805, 5202...","{1595392, 243714, 1644547, 466948, 102406, 499719, 546826, 419854, 1290254, 1556498, 1327123, 52...","{1343494, 1536007, 245766, 317450, 1597450, 118796, 319501, 118805, 471065, 503836, 1607712, 118...","{1089538, 116742, 1323024, 118805, 1646615, 1595415, 116761, 542748, 1587229, 1585183, 282656, 2...","{1601547, 118805, 1343510, 544789, 102430, 1251361, 548900, 528420, 542765, 247858, 178232, 1308..."
5,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1343494, 245766, 1323028, 247832, 1308713, 1316907, 1212475, 348223, 471108, 1321029, 1052743, ...","{1343494, 245766, 1085451, 1316875, 1433614, 669711, 1323024, 1331217, 1310734, 1323028, 1333269...","{745476, 1343494, 1308689, 1327123, 563221, 1343510, 471065, 284700, 41012, 178232, 471108, 5776...","{1338373, 163336, 1540628, 1336853, 1326614, 1502744, 280095, 1212475, 1347137, 523330, 547399, ..."
6,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1089538, 466948, 22